In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import random
from datetime import datetime
import time
import math

In [4]:
import torch
import torchvision
import torch.nn as nn
from torchvision.transforms import functional
from torchvision.transforms import v2

In [5]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
set_seed(500)

In [6]:
wd = os.getcwd()

## Import Swin_V2 Model

In [7]:
from torchvision.models import swin_v2_t, Swin_V2_T_Weights

In [8]:
swin_model = swin_v2_t(weights=Swin_V2_T_Weights.DEFAULT)
preprocess = Swin_V2_T_Weights.DEFAULT.transforms()

In [9]:
preprocess

ImageClassification(
    crop_size=[256]
    resize_size=[260]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)

## Load and Process Data

In [10]:
# Class representing a set
class WeedsDataset():
    def __init__(self, labels, img_dir, transform = None):
        self.labels = pd.read_csv(labels)
        self.img_dir = img_dir
        self.transform = transform
        self.classes = self.labels[['Label','Species']].drop_duplicates().sort_values(by = 'Label').reset_index(drop = True)['Species']
        self.data = []
        self.train = []
        self.validation = []
        self.test = []
        
    # Load entire dataset into memory
    def __loaddata__(self):
        for row in self.labels.itertuples():
            filename = row.Filename
            label = torch.tensor(row.Label)
            img_path = os.path.join(self.img_dir, filename)
            image = self.transform(torchvision.io.read_image(img_path))
            self.data.append([image,label])
        del self.labels
        print('Data has been loaded')

    # Split into train/validation/test
    def __split__(self):
        random.shuffle(self.data)
        n = len(self.data)
        n_train = round(n * 0.8)
        n_valid = round(n * 0.1)
        
        self.train = self.data[:n_train]
        self.valid = self.data[n_train:n_train + n_valid]
        self.test = self.data[n_train + n_valid:]
        del self.data
        
        print('Data has been split')
        print('Size of training set is {}'.format(len(self.train)))
        print('Size of validation set is {}'.format(len(self.valid)))
        print('Size of test set is {}'.format(len(self.test)))

    # Augment with rotations and gaussian noise
    def __augment__(self, frac):
        n = round(len(self.train) * frac / 5)
        rotate_90 = random.sample(self.train, n)
        for row in rotate_90:
            image = row[0]
            row[0] = functional.rotate(image, 90)
        rotate_180 = random.sample(self.train, n)
        for row in rotate_180:
            image = row[0]
            row[0] = functional.rotate(image, 180)
        rotate_270 = random.sample(self.train,n)
        for row in rotate_270:
            image = row[0]
            row[0] = functional.rotate(image, 270)
        self.train += rotate_90 + rotate_180 + rotate_270
        gaussian = v2.GaussianNoise(0,0.08,False)
        noise = random.sample(self.train, 2*n)
        for row in noise:
            image = row[0]
            row[0] = gaussian(image)
        self.train += noise
        
        print('Data has been augmented')
        print('Size of training set is {}'.format(len(self.train)))
        
    # Get item from sets, format is (image,label)
    def __getitem__(self, idx, split):
        if split == 'train':
            item = self.train[idx]
        elif split == 'valid':
            item = self.valid[idx]
        elif split == 'test':
            item = self.test[idx]
        return item[0], item[1]

In [11]:
# Run functions to load and process data
weeds = WeedsDataset(rf'{wd}\mini_labels.csv', rf'{wd}\images', preprocess)
weeds.__loaddata__()
weeds.__split__()
weeds.__augment__(0.5)

Data has been loaded
Data has been split
Size of training set is 72
Size of validation set is 9
Size of test set is 9
Data has been augmented
Size of training set is 107


In [12]:
# Functions to convert the label back into the class
def convert_label(label, dataset):
    return dataset.classes[label.item()]

def convert_labels(labels,dataset):
    lst = []
    for label in labels:
        data_class = convert_label(label,dataset)
        lst.append(data_class)
    return lst

## Load Data

In [13]:
# Sends image and label tensors to gpu
def get_batch(dataset, current_index, batch_size):
    images,labels = zip(*dataset[current_index:current_index+batch_size])
    return torch.stack(images, dim = 0), torch.stack(labels)

In [14]:
# Check that everything is working properly
def matplotlib_imshow(img, one_channel=False):
    if one_channel:
        img = img.mean(dim=0)
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    if one_channel:
        plt.imshow(npimg, cmap="Greys")
    else:
        plt.imshow(np.transpose(npimg, (1, 2, 0)))

# Create a grid from the images and show them
#images,labels = get_batch(weeds.train, 1000, 32)
#img_grid = torchvision.utils.make_grid(images.cpu())
#matplotlib_imshow(img_grid, one_channel=True)
#print(convert_labels(labels.cpu(),weeds))

## Replacing Classification Head

In [15]:
for param in swin_model.parameters():
    param.requires_grad = False
layers = []
layers.append(nn.Linear(in_features = 768, out_features = 256, bias = True))
layers.append(nn.ReLU())
layers.append(nn.Linear(in_features = 256, out_features = 9, bias = True))
swin_model.head = nn.Sequential(*layers)

In [16]:
#info.summary(swin_model)

## Training

## Evaluation

In [17]:
# Lowest validation loss is 0.2069510916610145 at epoch 91

In [18]:
swin_model.load_state_dict(torch.load('finalmodel\model_20250222_211532_91', weights_only = True, map_location = torch.device('cpu')))

<>:1: SyntaxWarning: invalid escape sequence '\m'
<>:1: SyntaxWarning: invalid escape sequence '\m'
C:\Users\simzi\AppData\Local\Temp\ipykernel_8784\1757134040.py:1: SyntaxWarning: invalid escape sequence '\m'
  swin_model.load_state_dict(torch.load('finalmodel\model_20250222_211532_91', weights_only = True, map_location = torch.device('cpu')))


<All keys matched successfully>

In [19]:
def evaluate(batch_size):
    start = time.perf_counter()
    total = 0
    correct = 0
    batches = math.floor(len(weeds.test)/batch_size)
    current_index = 0
    
    for i in range(batches):
        inputs,labels = get_batch(weeds.train, current_index, batch_size)
        current_index += batch_size
        prediction = torch.argmax(swin_model(inputs), dim = 1)
        correct += sum(prediction == labels).item()
        total += batch_size

    remainder = len(weeds.test)%batch_size
    if remainder != 0:
        inputs,labels = get_batch(weeds.train, current_index, remainder)
        prediction = torch.argmax(swin_model(inputs), dim = 1)
        correct += sum(prediction == labels).item()
        total += remainder

    accuracy = 100*correct/total
    print ('Accuracy is {}%'.format(accuracy))
    end = time.perf_counter()
    print('Time taken is {}'.format(end-start))
    
    return 

In [20]:
with torch.no_grad():
    swin_model.eval()
    evaluate(20)

Accuracy is 88.88888888888889%
Time taken is 2.746477500011679


In [21]:
weeds.train[0][1] # actual

tensor(6)

In [22]:
torch.argmax(swin_model(get_batch(weeds.train,0,1)[0])) # prediction

tensor(6)

In [23]:
# Accuracy is 92.58%

In [24]:
torch.save(swin_model, 'trained_model\weedsv1.pt')

<>:1: SyntaxWarning: invalid escape sequence '\w'
<>:1: SyntaxWarning: invalid escape sequence '\w'
C:\Users\simzi\AppData\Local\Temp\ipykernel_8784\3095721756.py:1: SyntaxWarning: invalid escape sequence '\w'
  torch.save(swin_model, 'trained_model\weedsv1.pt')
C:\Users\simzi\AppData\Local\Temp\ipykernel_8784\3095721756.py:1: SyntaxWarning: invalid escape sequence '\w'
  torch.save(swin_model, 'trained_model\weedsv1.pt')


RuntimeError: Parent directory trained_model does not exist.

## Evaluation for F1 Score

In [ ]:
import numpy as np
import torch
import time
import math
from sklearn.metrics import confusion_matrix

# Formula for F1: (2 * FP) / (2 * TP + FP + FN) OR 2 * (Precision * Recall) / (Precision + Recall)
# Formula for Precision: TP / (TP + FP)
# Formula for Recall: TP / (TP + FN)

num_classes = len(weeds.classes)
num_classes # Verify if there are 9 classes

def evaluate_f1(batch_size):
    start = time.perf_counter()
    total = 0
    correct = 0
    batches = math.floor(len(weeds.test) / batch_size)
    current_index = 0
    
    all_preds = []
    all_labels = []

    # Split into batches
    for i in range(batches):
        inputs, labels = get_batch(weeds.train, current_index, batch_size)
        current_index += batch_size
        prediction = torch.argmax(swin_model(inputs), dim=1)
        
        correct += sum(prediction == labels).item()
        total += batch_size

        all_preds.extend(prediction.numpy())
        all_labels.extend(labels.numpy())

    # Handle remainder batch
    remainder = len(weeds.test) % batch_size
    if remainder != 0:
        inputs, labels = get_batch(weeds.train, current_index, remainder)
        prediction = torch.argmax(swin_model(inputs), dim=1)
        
        correct += sum(prediction == labels).item()
        total += remainder

        all_preds.extend(prediction.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

    accuracy = 100 * correct / total
    print('Accuracy is {}%'.format(accuracy))

    # Compute Micro-Averaged F1-Score
    num_classes = len(weeds.classes) 
    conf_matrix = confusion_matrix(all_labels, all_preds, labels=np.arange(num_classes))
    print(conf_matrix)

    total_TP = 0
    total_FP = 0
    total_FN = 0

    # Generate confuction matriz
    for i in range(num_classes):
        TP = conf_matrix[i, i]  # True Positives
        FP = np.sum(conf_matrix[:, i]) - TP  # False Positives
        FN = np.sum(conf_matrix[i, :]) - TP  # False Negatives

        total_TP += TP
        total_FP += FP
        total_FN += FN

    micro_precision = total_TP / (total_TP + total_FP) if (total_TP + total_FP) > 0 else 0
    micro_recall = total_TP / (total_TP + total_FN) if (total_TP + total_FN) > 0 else 0
    micro_f1 = (2 * micro_precision * micro_recall) / (micro_precision + micro_recall) if (micro_precision + micro_recall) > 0 else 0

    print(f"Micro Precision: {micro_precision:.4f}")
    print(f"Micro Recall: {micro_recall:.4f}")
    print(f"Micro F1-score: {micro_f1:.4f}")

    end = time.perf_counter()
    print('Time taken is {}'.format(end - start))


In [ ]:
with torch.no_grad():
    swin_model.eval()
    evaluate_f1(20)

## Augmentation with ColorJitter

In [ ]:
import torch
import numpy as np
import torchvision.transforms as transforms
import time
import math
import matplotlib.pyplot as plt
%matplotlib inline

# Define the ColorJitter and Gaussian transformation
color_jitter = transforms.ColorJitter(brightness = 0.3, contrast = 0.2, saturation = 0.3)
gaussian = v2.GaussianNoise(0, 0.08, False)

def visualize_images(original, augmented):
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))

    # Convert tensor images to numpy arrays for visualization
    original_img = original.permute(1, 2, 0).numpy()  # Convert from C x H x W to H x W x C
    augmented_img = augmented.permute(1, 2, 0).numpy()

    # Normalize the images to [0, 1] range if required (since they may be in [0, 255] or [-1, 1] range)
    original_img = np.clip(original_img, 0, 1)
    augmented_img = np.clip(augmented_img, 0, 1)

    # Display the images
    axes[0].imshow(original_img)
    axes[0].set_title('Original Image')
    axes[0].axis('off')

    axes[1].imshow(augmented_img)
    axes[1].set_title('Augmented Image')
    axes[1].axis('off')

    plt.show() 
    plt.close(fig) 

def augment_cj(batch_size):
    start = time.perf_counter()
    total = 0
    correct = 0
    batches = math.floor(len(weeds.test) / batch_size)
    current_index = 0

    all_preds = []  
    all_labels = []

    for i in range(batches):
        # Get batch of data
        inputs, labels = get_batch(weeds.train, current_index, batch_size)
        current_index += batch_size

        # Check the range before normalization
        print(f"Before normalization: min={inputs.min()}, max={inputs.max()}")

        # Normalize the entire batch (to [0, 1] range)
        eps = 1e-8  
        inputs = (inputs - inputs.min()) / (inputs.max() - inputs.min() + eps)

        # Check after normalization
        print(f"After normalization: min={inputs.min()}, max={inputs.max()}")

        # Select the first image from the batch for visualization
        original_image = inputs[0]  
        
        # Apply ColorJitter to augment the images
        cj_image = color_jitter(original_image) 
        augmented_image = gaussian(cj_image)

        # Visualize the original vs augmented image
        visualize_images(original_image, augmented_image)
        
        # Check if any pixel values in inputs are negative
        if (inputs < 0).any():
            print(f"Warning: Negative pixel values detected in inputs at batch {i}")
            negative_pixel_indices = np.where(inputs.numpy() < 0)
            print(f"Negative pixel indices at batch {i}: {negative_pixel_indices}")
        
        # Forward pass through the model
        prediction = torch.argmax(swin_model(inputs), dim=1)

        # Check for negative values in the predictions 
        if (prediction < 0).any():
            print(f"Warning: Negative values detected in predictions at batch {i}")
            
        # Store the predictions and labels for further analysis
        all_preds.extend(prediction.numpy())
        all_labels.extend(labels.numpy())

    # Check for negative values in all predictions and labels
    if any(val < 0 for val in all_preds):
        print("Warning: Negative values detected in all_preds!")
    if any(val < 0 for val in all_labels):
        print("Warning: Negative values detected in all_labels!")
    else:
        print("There are no negative values detected")
        
    end = time.perf_counter()
    print('Time taken is {}'.format(end - start))

    return all_preds, all_labels


In [ ]:
with torch.no_grad():
    swin_model.eval()
    augment_cj(1)

## Augmentation with Cropping

In [ ]:
import torch
import numpy as np
import torchvision.transforms as transforms
import time
import math
import matplotlib.pyplot as plt
%matplotlib inline

# Define the RandomCrop and Gaussian transformation
random_crop = transforms.RandomCrop(size=(100, 100)) 
gaussian = v2.GaussianNoise(0, 0.08, False)

def augment_crop(batch_size):
    start = time.perf_counter()
    total = 0
    correct = 0
    batches = math.floor(len(weeds.test) / batch_size)
    current_index = 0

    all_preds = []  
    all_labels = []

    for i in range(batches):
        # Get batch of data
        inputs, labels = get_batch(weeds.train, current_index, batch_size)
        current_index += batch_size

        # Check the range before normalization
        print(f"Before normalization: min={inputs.min()}, max={inputs.max()}")

        # Normalize the entire batch (to [0, 1] range)
        eps = 1e-8  
        inputs = (inputs - inputs.min()) / (inputs.max() - inputs.min() + eps)

        # Check after normalization
        print(f"After normalization: min={inputs.min()}, max={inputs.max()}")

        # Select the first image from the batch for visualization
        original_image = inputs[0]  

        # Apply RandomCrop to augment the image
        cropped_image = random_crop(original_image) 
        augmented_image = gaussian(cropped_image)

        # Visualize the original vs augmented image
        visualize_images(original_image, augmented_image)
        
        # Check if any pixel values in inputs are negative
        if (inputs < 0).any():
            print(f"Warning: Negative pixel values detected in inputs at batch {i}")
            negative_pixel_indices = np.where(inputs.numpy() < 0)
            print(f"Negative pixel indices at batch {i}: {negative_pixel_indices}")
        
        # Forward pass through the model
        prediction = torch.argmax(swin_model(inputs), dim=1)

        # Check for negative values in the predictions 
        if (prediction < 0).any():
            print(f"Warning: Negative values detected in predictions at batch {i}")
            
        # Store the predictions and labels for further analysis
        all_preds.extend(prediction.numpy())
        all_labels.extend(labels.numpy())

    # Check for negative values in all predictions and labels
    if any(val < 0 for val in all_preds):
        print("Warning: Negative values detected in all_preds!")
    if any(val < 0 for val in all_labels):
        print("Warning: Negative values detected in all_labels!")
    else:
        print("There are no negative values detected")
        
    end = time.perf_counter()
    print('Time taken is {}'.format(end - start))

    return all_preds, all_labels


In [ ]:
with torch.no_grad():
    swin_model.eval()
    augment_crop(1)

## Inference Time Per Image

In [30]:
import time


def inference_time_per_image(model, dataset, num_samples):
    model.eval()
    indices = np.random.choice(len(dataset), num_samples, replace=False)
    total_time = 0

    print(f"\n🔍 Measuring inference time for {num_samples} images...\n")

    for i, idx in enumerate(indices):
        image, label = dataset[idx]

        # Add batch dimension since model expects batches
        input_tensor = image.unsqueeze(0)

        start_time = time.perf_counter()

        # Run inference
        with torch.no_grad():
            _ = model(input_tensor)

        end_time = time.perf_counter()
        elapsed_time = end_time - start_time
        total_time += elapsed_time

        print(f"Image {i+1} | Inference Time: {elapsed_time:.5f} seconds")

    avg_time = total_time / num_samples
    print(f"\nAverage Inference Time: {avg_time:.5f} seconds per image")

    return avg_time

    

In [33]:
# Run timing for inference on a few test images
average_time = inference_time_per_image(swin_model, weeds.test, num_samples=len(weeds.test))


🔍 Measuring inference time for 9 images...

Image 1 | Inference Time: 0.43797 seconds
Image 2 | Inference Time: 0.31151 seconds
Image 3 | Inference Time: 0.30753 seconds
Image 4 | Inference Time: 0.30054 seconds
Image 5 | Inference Time: 0.28023 seconds
Image 6 | Inference Time: 0.30530 seconds
Image 7 | Inference Time: 0.54159 seconds
Image 8 | Inference Time: 0.29613 seconds
Image 9 | Inference Time: 0.28613 seconds

Average Inference Time: 0.34077 seconds per image
